In [125]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
# ignore warnings
import warnings
import math

warnings.filterwarnings('ignore')

In [126]:
column_names = [
"address",
"price",
"gross_tax",
"strata_maintenance_fees",
"bedrooms",
"bathrooms",
"property_type",
"property_age",
"title",
"style",
"heating_type",
"feature",
"amenities",
"appliances",
"community",
"days_on_rew",
"property_views",
"mls®_number",
"source",
'frontage', 
'lot_size', 
'year_built', 
'depth',
'half_bathrooms'
]



In [127]:

def get_df(file_path, apply_columns=True):

    sys.path.append(file_path)

    data = pd.read_csv(file_path, on_bad_lines='skip')

    # data.dropna(inplace=True)
    if apply_columns:
        data.columns = column_names
    return data



folder = "../data/raw_2023_04_19/"

filenames = {
    "burnaby": "burnaby_real_estate_data.csv",
    "langley":"langley_real_estate_data.csv",
    "richmond":"richmond_real_estate_data.csv",
    "maple_ridge":"maple-ridge_real_estate_data.csv",
    "surrey":"surrey_real_estate_data.csv",
    "new_west": "new-westminster_real_estate_data.csv",
    "vancouver":"vancouver_real_estate_data.csv",
    "coquitlam":"coquitlam_real_estate_data.csv",
    "north_vancouver":"north-vancouver_real_estate_data.csv",
    "west_vancouver":"west-vancouver_real_estate_data.csv",
    "delta":"delta_real_estate_data.csv",
    "pitt_meadows":"pitt-meadows_real_estate_data.csv",
    "kelowna":"kelowna_real_estate_data.csv",
    "port_coquitlam":"port-coquitlam_real_estate_data.csv"
}

dataframes = {}

for k,v in filenames.items():
    dataframes[k] = get_df(folder + v)

dataframes.keys()



dict_keys(['burnaby', 'langley', 'richmond', 'maple_ridge', 'surrey', 'new_west', 'vancouver', 'coquitlam', 'north_vancouver', 'west_vancouver', 'delta', 'pitt_meadows', 'kelowna', 'port_coquitlam'])

In [128]:
# get neighbourhoods per city

neighbourhoods_city = {}

for city, df in dataframes.items():
    
    neighbourhoods = df['community'].unique()
    for neighbourhood in neighbourhoods:

        neighbourhoods_city[neighbourhood] = city


In [129]:
# find out which city the row is in
clean_combined_data = get_df(file_path=folder + "clean_combined_data.csv", apply_columns=False)

clean_combined_data['city'] = clean_combined_data['community'].apply(lambda x: neighbourhoods_city.get(x, 'unknown'))

# Select Features

In [130]:
# Select the specified columns
# selected_columns = ['address', 'price', 'gross_tax', 'bedrooms',
#                     'bathrooms', 'property_type', 
#                     'community', 'lot_size', 'half_bathrooms', 'lot_width', 'lot_length',
#                     'age', 'bungalow', 'storey', 'basement', 'laneway_house', 'garage',
#                     'split_entry', 'city']

selected_columns = [
    'price', 
    'city',
    'gross_tax',
    'community', 
    'bedrooms',
    'bathrooms', 
    'lot_width',
    'lot_length',
    'age',
    'bungalow',
    'storey', 
    'basement', 
    'laneway_house',
    'garage',
    'split_entry',
]

df = clean_combined_data[selected_columns]



# Code Cleanup

In [131]:
# Drop 0 Rows
# df = df[df['gross_tax'] != 0]
# df = df[df['lot_size'] != 0]
df = df[df['lot_width'] != 0]
df = df[df['lot_length'] != 0]
# before (2104, 294)
df = df.dropna()
# after (1379, 294)

# round price for better prediction
df['price'] = df['price'].apply(lambda x: math.ceil(x / 500000) * 500000)

df.shape


(1382, 15)

In [132]:
# Identify the categorical columns
categorical_columns = df.select_dtypes(include=['object']).columns

# Apply one-hot encoding to the categorical columns
df = df.drop_duplicates()

df = pd.get_dummies(df, columns=categorical_columns)
df

,price,gross_tax,bedrooms,bathrooms,lot_width,lot_length,age,bungalow,storey,basement,...,community_whalley,community_whitby estates,community_white rock,community_whonnock,community_whytecliff,community_willingdon heights,community_willoughby heights,community_woodland acres,community_woodlands-sunshine-cascade,community_woodwards
0,3500000,4504,7.0,7.0,63.0,63.0,0.0,0,2,1,...,0,0,0,0,0,0,0,0,0,0
4,2500000,5129,6.0,2.0,72.0,72.0,64.0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
5,2000000,3750,5.0,2.0,122.0,122.0,67.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9,3000000,4129,7.0,6.0,68.0,68.0,-1.0,0,2,1,...,0,0,0,0,0,0,0,0,0,0
10,2000000,4935,5.0,2.0,57.0,57.0,31.0,0,2,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2185,2500000,6531,5.0,3.0,69.0,69.0,46.0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
2186,3000000,7579,5.0,4.0,50.0,50.0,2.0,0,3,1,...,0,0,0,0,0,0,0,0,0,0
2187,3500000,9413,9.0,6.0,57.0,57.0,11.0,0,2,1,...,0,0,0,0,0,0,0,0,0,0
2188,3000000,4560,7.0,4.0,50.0,50.0,0.0,0,2,1,...,0,0,0,0,0,0,0,0,0,0


# Random Forest

In [133]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression


# Scale features
scaler = StandardScaler()

# split x and y
X = df.drop(['price'], axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1)
train_data = X_train.join(y_train)
test_data = X_test.join(y_test)

X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.fit_transform(X_test)



In [134]:
from sklearn.ensemble import RandomForestClassifier
forest_s = RandomForestClassifier()
forest_s.fit(X_train_s, y_train)
forest_s.score(X_test_s, y_test)

0.3669064748201439

# Neural Network

## Training

In [135]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import math
import tensorflow as tf
import keras


data = df

# Define the target variable (price) and feature columns
target = 'price'
features = data.drop(columns=[target]).columns

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data[features], data[target], test_size=0.2, random_state=42)

# Normalize the data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the neural network model
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=500, batch_size=32, verbose=1, callbacks=[keras.callbacks.EarlyStopping(patience=3)])


Epoch 1/500
35/35 [==============================] - 0s 1ms/step - loss: 16743881768960.0000 - accuracy: 0.0000e+00
Epoch 2/500
35/35 [==============================] - 0s 980us/step - loss: 16743176077312.0000 - accuracy: 0.0000e+00
Epoch 3/500
35/35 [==============================] - 0s 1ms/step - loss: 16722181488640.0000 - accuracy: 0.0000e+00
Epoch 4/500
35/35 [==============================] - 0s 1ms/step - loss: 16490497572864.0000 - accuracy: 0.0000e+00
Epoch 5/500
35/35 [==============================] - 0s 1ms/step - loss: 15125688877056.0000 - accuracy: 0.0000e+00
Epoch 6/500
35/35 [==============================] - 0s 1ms/step - loss: 10761767223296.0000 - accuracy: 0.0000e+00
Epoch 7/500
35/35 [==============================] - 0s 1ms/step - loss: 5872562470912.0000 - accuracy: 0.0000e+00
Epoch 8/500
35/35 [==============================] - 0s 1ms/step - loss: 5119224578048.0000 - accuracy: 0.0000e+00
Epoch 9/500
35/35 [==============================] - 0s 1ms/step - loss:

## Evaluation

In [136]:

# Evaluate the model
loss = model.evaluate(X_test_scaled, y_test)
mse = loss[0]
formatted_mse = "{:,.2f}".format(mse)
print(f'Mean Squared Error: {formatted_mse}')

rmse = math.sqrt(mse)
formatted_mse = "{:,.2f}".format(rmse)
print(f'Root Mean Squared Error: {formatted_mse}')


9/9 [==============================] - 0s 911us/step - loss: 1472778076160.0000 - accuracy: 0.0000e+00
Mean Squared Error: 1,472,778,076,160.00
Root Mean Squared Error: 1,213,580.68
9/9 [==============================] - 0s 722us/step


## Predict

In [ ]:

# Make predictions
predictions = model.predict(X_test_scaled)

In [137]:
# Concatenate predicted and actual values
results = np.concatenate((predictions, y_test.values.reshape(-1, 1)), axis=1)

# Create DataFrame
results_df = pd.DataFrame(results, columns=['Predicted', 'Actual'])
results_df['Difference'] = abs(results_df['Actual'] - results_df['Predicted'])

formated_df = results_df.copy()
formated_df['Predicted'] = results_df['Predicted'].apply(lambda x: '{:,.2f}'.format(x))
formated_df['Actual'] = results_df['Actual'].apply(lambda x: '{:,.2f}'.format(x))
formated_df['Difference'] = results_df['Difference'].apply(lambda x: '{:,.2f}'.format(x))

mean = results_df['Difference'].mean()
formatted_mean = "{:,.2f}".format(mean)
print("average difference",  formatted_mean)
# Print results
formated_df.head()


average difference 814,752.64


,Predicted,Actual,Difference
0,"2,608,669.50","2,500,000.00","108,669.50"
1,"1,956,619.25","2,500,000.00","543,380.75"
2,"2,371,096.50","3,500,000.00","1,128,903.50"
3,"1,716,985.50","2,000,000.00","283,014.50"
4,"2,022,405.38","2,000,000.00","22,405.38"


In [138]:
# Concatenate predicted and actual values
results = np.concatenate((predictions, y_test.values.reshape(-1, 1)), axis=1)

# Create DataFrame
results_df = pd.DataFrame(results, columns=['Predicted', 'Actual'])

# Print results
results_df.head()

,Predicted,Actual
0,2608669.500,2500000.0
1,1956619.250,2500000.0
2,2371096.500,3500000.0
3,1716985.500,2000000.0
4,2022405.375,2000000.0


In [140]:
tf.saved_model.save(model, 'housingNNmodel')

INFO:tensorflow:Assets written to: housingNNmodel/assets
